In [0]:
# import
from py4j.protocol import Py4JJavaError

In [0]:
# ——————————————————————————————
#  COMMON SETUP (pro všechny notebooky)
# ——————————————————————————————
try:
    env = dbutils.widgets.get("pipeline.env")
except:
    env = "dev"

catalog = "principal_lab_db"
silver_schema = f"{env}_silver"
gold_schema = f"{env}_gold"

# Katalog
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {silver_schema}")

In [0]:
# Funkce pro logování

def run_gold_view_queries(view_queries):
    for i, query in enumerate(view_queries):
        try:
            # Získání názvu view
            view_name = query.split("VIEW")[1].split("AS")[0].strip()
            print(f"[{i+1}/{len(view_queries)}] Vytvářím view: {view_name}")

            # Spuštění
            spark.sql(query)

            print(f"View '{view_name}' bylo úspěšně vytvořeno.\n")

        except Py4JJavaError as e:
            msg = e.java_exception.getMessage()
            print(f"Chyba při vytváření view '{view_name}': {msg}\n")

        except Exception as e:
            print(f"Chyba u view '{view_name}': {str(e)}\n")

In [0]:
# ——————————————————————————————
#  CREATE GOLD VIEW
# ——————————————————————————————
gold_view_queries = []

# agent_performance_view
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.agent_performance_view AS
SELECT
    a.agent_id,
    a.first_name,
    a.last_name,
    COUNT(DISTINCT p.policy_id) AS num_policies,
    ROUND(SUM(pt.premium_amount), 2) AS total_premium,
    COUNT(DISTINCT c.claim_id) AS num_claims,
    ROUND(SUM(c.amount), 2) AS total_claim_amount
FROM {catalog}.{silver_schema}.dim_agents_mask a
LEFT JOIN {catalog}.{silver_schema}.dim_policies_mask p USING (agent_id)
LEFT JOIN {catalog}.{silver_schema}.fact_premium_transactions_mask pt USING (policy_id)
LEFT JOIN {catalog}.{silver_schema}.fact_claims_mask c USING (policy_id)
GROUP BY a.agent_id, a.first_name, a.last_name
""")

# agen_language_distribution
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.agent_language_distribution AS
SELECT
  lang AS language_code,
  COUNT(DISTINCT agent_id) AS num_agents
FROM {catalog}.{silver_schema}.dim_agents_mask
LATERAL VIEW explode(languages) AS lang
GROUP BY lang
ORDER BY num_agents DESC
""")

# customer_income_distribution
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.customer_income_distribution AS
select  
first_name
, last_name
, email
, address
, sum(income) as income
, case 
    when sum(income) < 250000 then 'low'
    when sum(income) < 1000000 then 'medium'
    else 'high' 
    end as income_level
from {catalog}.{silver_schema}.dim_customers_mask
where `__END_AT` is null
group by all
order by income desc
""")

# agents_with_flagged_payments
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.agents_with_flagged_payments AS
with agent as (
    select
        agent_id
        , payment_date
        , count(*) as payment_count
    from {catalog}.{silver_schema}.fact_premium_transactions_mask
    where paid_flag = true
      and payment_date is not null
    group by agent_id, payment_date
),
avg_payment as (
    select
        payment_date
        , avg(payment_count) as avg_cnt
    from agent
    group by payment_date
),
flagged_payments as (
    select 
        fpt.agent_id
        , count(*) as payment_count
        , ap.avg_cnt
        , 'neobvykly pocet plateb' as payment_red_flag
    from {catalog}.{silver_schema}.fact_premium_transactions_mask fpt
    left join avg_payment ap on fpt.payment_date = ap.payment_date
    where fpt.paid_flag = true
      and fpt.payment_date is not null
    group by fpt.agent_id, ap.avg_cnt
    having count(*) > 2 * ap.avg_cnt
),
agent_totals as (
    select
        agent_id
        , count(*) as payment_count
    from {catalog}.{silver_schema}.fact_premium_transactions_mask
    where paid_flag = true
    group by agent_id
),
percentile_val as (
    select
        percentile_approx(payment_count, 0.95) as p95
    from agent_totals
),
unusual_by_volume as (
    select distinct
        a.agent_id
        , 'vyssi objem prijatych platieb' as payment_red_flag
    from agent_totals a
    join percentile_val p on a.payment_count > p.p95
),
unusual_by_count as (
    select distinct
        fp.agent_id
        , fp.payment_red_flag
    from flagged_payments fp
)
select 
    u.agent_id
    , da.first_name
    , da.last_name
    , u.payment_red_flag
from unusual_by_count u
left join {catalog}.{silver_schema}.dim_agents da on u.agent_id = da.agent_id
union
select 
    u.agent_id
    , da.first_name
    , da.last_name
    , u.payment_red_flag
from unusual_by_volume u
left join {catalog}.{silver_schema}.dim_agents da on u.agent_id = da.agent_id
""")

# coverage_distribution
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.coverage_distribution AS
WITH base AS (
  SELECT EXPLODE(coverages) as coverage_expl
  FROM {catalog}.{silver_schema}.dim_policies_mask
  WHERE __END_AT is null --actual rows
),pre_fin AS (
  SELECT coverage_expl as coverage_name
        ,COUNT(coverage_expl) as coverage_count
  FROM base
  GROUP BY coverage_expl
)
SELECT coverage_name
      ,coverage_count
      ,ROW_NUMBER() OVER (ORDER BY coverage_count DESC) AS coverage_position
FROM pre_fin
ORDER BY coverage_position ASC
""")

#daily_premium_trend
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.daily_premium_trend AS
WITH base (
      SELECT payment_date
            ,ROUND(SUM(premium_amount),2) AS daily_paymend
      FROM {catalog}.{silver_schema}.fact_premium_transactions_mask
      WHERE paid_flag = True
      GROUP BY payment_date
),pre_fin (
   SELECT payment_date
      ,daily_paymend
      ,LAG(daily_paymend,1,0) OVER(ORDER BY payment_date ASC) AS previous_day_payment
FROM base   
)
SELECT payment_date
      ,daily_paymend
      ,CASE
            WHEN previous_day_payment = 0 THEN NULL
            ELSE ROUND((daily_paymend - previous_day_payment)/previous_day_payment,2)
      END AS payment_ratio_act_day_vs_prev_day
FROM pre_fin                         
""")

#claim_ratio
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.claim_ratio_per_product AS
WITH claims AS (
  SELECT dpr.product_id
        ,dpr.product_name
        ,dpr.sub_product AS sub_product_name
        ,round(sum(fc.amount),2) AS claim_amount
  FROM {catalog}.{silver_schema}.fact_claims_mask fc 
  JOIN {catalog}.{silver_schema}.dim_policies_mask dp ON fc.policy_id = dp.policy_id AND dp.__END_AT is null
  JOIN {catalog}.{silver_schema}.dim_products_mask dpr ON dp.product_id = dpr.product_id
  GROUP BY dpr.product_id,dpr.product_name,dpr.sub_product
),prem_transact AS (
  SELECT dpr.product_id
        ,dpr.product_name
        ,dpr.sub_product
        ,ROUND(SUM(fpt.premium_amount),2) AS premium_amount
  FROM {catalog}.{silver_schema}.fact_premium_transactions_mask fpt
  JOIN {catalog}.{silver_schema}.dim_policies_mask dp ON fpt.policy_id = dp.policy_id AND dp.__END_AT is null AND fpt.paid_flag = True
  JOIN {catalog}.{silver_schema}.dim_products_mask dpr ON dp.product_id = dpr.product_id
  GROUP BY dpr.product_id,dpr.product_name,dpr.sub_product
)SELECT c.*
        ,p.premium_amount
        ,ROUND((c.claim_amount / p.premium_amount), 2) AS claim_ratio
FROM claims c join prem_transact p on c.product_id = p.product_id
ORDER BY claim_ratio DESC                      
""")

#daily_premium_by_region
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.daily_premium_by_region AS
WITH premium AS ( -- 
      SELECT  payment_date
            ,premium_amount
            ,agent_id
            ,snapshot_date
      FROM {catalog}.{silver_schema}.fact_premium_transactions_mask
      WHERE paid_flag = True
),agents AS (
      SELECT agent_id
            ,region
            ,__START_AT
            ,NVL(__END_AT,"2400-01-01") AS __END_AT
      FROM {catalog}.{silver_schema}.dim_agents_mask
)--inner join because some agents data from snapshots does not pass dq, so will be null in result 
SELECT p.payment_date
      ,a.region
      ,ROUND(SUM(p.premium_amount),2) AS daily_premium
FROM premium p
JOIN agents a ON p.agent_id = a.agent_id AND p.snapshot_date BETWEEN a.__START_AT AND a.__END_AT
GROUP BY p.payment_date,a.region
ORDER BY region asc,payment_date ASC
""")

#daily_premium_by_region
gold_view_queries.append(f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.top_customers_by_premium AS
WITH premium AS (
  SELECT policy_id
        ,snapshot_date
        ,premium_amount
  FROM {catalog}.{silver_schema}.fact_premium_transactions_mask
),policy AS (
  SELECT policy_id
        ,customer_id
        ,__START_AT
        ,NVL(__END_AT,'2400-01-01') AS __END_AT
  FROM {catalog}.{silver_schema}.dim_policies_mask
),customer AS (
    SELECT customer_id
            ,first_name
            ,last_name
    FROM {catalog}.{silver_schema}.dim_customers_mask
    WHERE __END_AT is null
),fin AS (
  SELECT c.customer_id
        ,c.first_name
        ,c.last_name
        ,ROUND(SUM(p.premium_amount),2) AS paid_premium
  FROM premium p 
  JOIN policy po ON p.policy_id = po.policy_id --because some policy are not open in dim - data inconsistency
                          AND p.snapshot_date BETWEEN po.__START_AT AND po.__END_AT
  LEFT JOIN customer c ON po.customer_id = c.customer_id 
  GROUP BY c.customer_id,c.first_name,c.last_name           
)SELECT customer_id
        ,first_name
        ,last_name
        ,paid_premium
FROM fin 
""")

In [0]:
run_gold_view_queries(gold_view_queries)